# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [142]:
# Use this cell to write your code for Task 1
import pandas as pd
df = pd.read_csv('house_sales.csv')
cities = {'Silvertown', 'Riverford', 'Teasdale', 'Poppleton'}
missing_city = df[~df['city'].isin(cities)]
missing_city = missing_city.shape[0]
missing_city

73

# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [143]:
# Use this cell to write your code for Task 2
import pandas as pd

def clean_house_data(df):
    df['city'] = df['city'].replace("--", None).fillna("Unknown")
    df['sale_price'] = df['sale_price'].dropna()
    df['sale_date'] = df['sale_date'].fillna("2023-01-01")
    df['months_listed'] = df['months_listed'].fillna(df['months_listed'].mean()).round(1)
    df['bedrooms'] = df['bedrooms'].fillna(round(df['bedrooms'].mean()))
    common_house_type = df['house_type'].mode()[0]
    df['house_type'] = df['house_type'].fillna(common_house_type)
    df['house_type'] = df['house_type'].replace({
        "Det.": "Detached",
        "Semi": "Semi-detached",
        "Terr.": "Terraced"})
    df['area'] = df['area'].astype(str).str.replace(" sq.m.", "").astype(float)
    df['area'] = df['area'].fillna(df['area'].mean()).round(1)

    print("Complete.")
    
    return df

df = pd.read_csv('house_sales.csv')
clean_data = clean_house_data(df)

clean_data.to_csv('clean_data.csv', index=False)
print(f"Complete. Saved as 'clean_data.csv'.") 

Complete.
Complete. Saved as 'clean_data.csv'.


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [144]:
# Use this cell to write your code for Task 3
import pandas as pd

df = pd.read_csv('clean_data.csv')

price_by_rooms = df.groupby('bedrooms')['sale_price'].agg(
        avg_price='mean', 
        var_price='var'
).reset_index()
price_by_rooms = price_by_rooms.round({'avg_price': 1, 'var_price': 1})
price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [145]:
# Use this cell to write your code for Task 4
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

train_data = pd.read_csv('train.csv')
validation_data = pd.read_csv('validation.csv')

X_train = train_data[['bedrooms']]
y_train = train_data['sale_price']


model = LinearRegression()
model.fit(X_train, y_train)

X_validation = validation_data[['bedrooms']]
predicted_prices = model.predict(X_validation)

base_result = pd.DataFrame({
    'house_id': validation_data['house_id'],
    'price': predicted_prices
})

base_result['price'] = base_result['price'].round(2)
base_result

,house_id,price
0,1331375,151438.23
1,1630115,227427.81
2,1645745,379406.97
3,1336775,151438.23
4,1888274,227427.81
...,...,...
295,1986255,379406.97
296,1896276,379406.97
297,1758223,227427.81
298,1752010,151438.23


In [146]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Preprocessing Function
def preprocess_data(df, is_train=True):
    df = df.copy()
    df = df.drop(columns=['sale_date'])
    df = pd.get_dummies(df, columns=['city', 'house_type'], drop_first=True)
    
    if is_train:
        X = df.drop(columns=['sale_price', 'house_id'])
        y = df['sale_price']
        return X, y
    else:
        X = df.drop(columns=['house_id'])
        house_ids = df['house_id']
        return X, house_ids

train = pd.read_csv("train.csv")
validation = pd.read_csv("validation.csv")

# Preprocessing train and validation datasets
X_train, y_train = preprocess_data(train, is_train=True)
X_validation, house_ids = preprocess_data(validation, is_train=False)

baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)

# Predicting on the validation set
baseline_predictions = baseline_model.predict(X_validation)

base_result = pd.DataFrame({
    'house_id': validation_data['house_id'],
    'price': baseline_predictions
})

base_result['price'] = base_result['price'].round(2)
base_result

,house_id,price
0,1331375,121527.83
1,1630115,304386.63
2,1645745,384760.10
3,1336775,123976.27
4,1888274,271186.20
...,...,...
295,1986255,351916.47
296,1896276,370138.97
297,1758223,259024.12
298,1752010,169120.16


In [147]:
# Calculating RMSE for the baseline model
baseline_rmse = np.sqrt(mean_squared_error(house_ids, baseline_predictions, squared=False))
baseline_rmse

1159.9142104296477

# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [148]:
# Standardized
# Initialize and fit the StandardScaler
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(X_validation)

comparison_model = LinearRegression()
comparison_model.fit(X_train_scaled, y_train)

# Predicting on the validation set
comparison_model_predictions = comparison_model.predict(X_validation_scaled)

comparison_result = pd.DataFrame({
    'house_id': validation_data['house_id'],
    'price': comparison_model_predictions
})

comparison_result['price'] = comparison_result['price'].round(2)
comparison_result

,house_id,price
0,1331375,121527.83
1,1630115,304386.63
2,1645745,384760.10
3,1336775,123976.27
4,1888274,271186.20
...,...,...
295,1986255,351916.47
296,1896276,370138.97
297,1758223,259024.12
298,1752010,169120.16


In [149]:
# Calculating RMSE for the baseline model
comparison_model_RMSE = np.sqrt(mean_squared_error(house_ids, comparison_model_predictions, squared=False))
comparison_model_RMSE

1159.9142104296477